In [1]:
#%% import libraries 
import numpy as np
import os
from scipy import io
import re
import h5py
import math
import matplotlib.pyplot as plt
# from global_scripts import spiketools
import gc
# import torch
from tqdm import tqdm
import warnings
from typing import Tuple
import pickle
import math
import sys
# Change the current directory
os.chdir("/Users/niloughazavi/Documents/GitHub/RetinaPredictors-main")

# Verify the change
print("Current working directory:", os.getcwd())

from model.load_savedModel import *
from model.utils_si import *
import model.data_handler
from model.data_handler import load_data, prepare_data_cnn2d, prepare_data_cnn3d, prepare_data_convLSTM, prepare_data_pr_cnn2d,merge_datasets,isintuple
from model.data_handler_mike import load_h5Dataset
from model.performance import getModelParams, model_evaluate,model_evaluate_new,paramsToName, get_weightsDict, get_weightsOfLayer,estimate_noise
from model import metrics
from model import featureMaps
from model.models import modelFileName
from model.train_model import chunker
import model.gradient_tools
from model.featureMaps import spatRF2DFit, get_strf, decompose
from model.train_model import chunker
from model.train_model import chunker

Current working directory: /Users/niloughazavi/Documents/GitHub/RetinaPredictors-main
Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


In [2]:

sys.path.append('/Users/niloughazavi/Documents/GitHub/CNN_data')

from WN_functions import *

Current working directory: /Users/niloughazavi/Documents/GitHub/RetinaPredictors-main


In [8]:
# %% exp details
expDate = '20240229C'
UP_SAMP = 0
bin_width = 8   # ms
sig = 4#8 # 2   #2 for 16 ms; 4 for 8 ms; 8 for 4 ms; 20 for 1 ms


CONVERT_RSTAR = True
NORM_STIM = 1
NORM_RESP = True


path_raw = os.path.join('/Users/niloughazavi/Documents/GitHub/Gradients/RGC_Selective_Simulation/White_Noise/analyses/data_mike_noise',expDate,'raw')
path_db = os.path.join('/Users/niloughazavi/Documents/GitHub/Gradients/RGC_Selective_Simulation/White_Noise/analyses/data_mike_noise',expDate,'datasets')
path_save = os.path.join('/Users/niloughazavi/Documents/GitHub/Gradients/RGC_Selective_Simulation/White_Noise/analyses/data_mike_noise',expDate,'datasets')
if CONVERT_RSTAR==True:
    fname_save = os.path.join(path_save,(expDate+'_dataset_CB_CORR_'+'allLightLevels'+'_'+str(bin_width)+'ms'+'_Rstar.h5'))
else:
    fname_save = os.path.join(path_save,(expDate+'_dataset_CB_CORR_'+'allLightLevels'+'_'+str(bin_width)+'ms.h5'))


if not os.path.exists(path_save):
    os.mkdir(path_save)

select_colChan = 0
stride = 2
depth = 61 # You'll need to increase this for the mesopic and scotopic noise runs because the kinetics are slower.

# Load the data.
lightLevel_text = ['scotopic','mesopic','photopic']
lightLevel_mul = [0.5,50,6000]

In [ ]:
## read the text files

mesopic_cell_dove = {
    'OffP': [5,49,52,130,137, 138, 258, 316, 360, 444, 498, 521, 575, 594, 632, 648, 688, 713, 721, 741, 785, 998],
    'OnP': [12, 20,140, 172, 196, 207, 223, 313, 342, 445, 446, 507, 533, 571, 630, 651, 752, 764, 788,969, 980]
}




mesopic_off_p = mesopic_cell_dove['OffP']
mesopic_on_p = mesopic_cell_dove['OnP']


# only parasol cells 
uname_all = list()
ctr = 0
for i in mesopic_on_p:
    ctr+=1
    uname_rgb = 'on_par_%03d'%ctr
    uname_all.append(uname_rgb)
    
ctr = 0
for i in mesopic_off_p:
    ctr+=1
    uname_rgb = 'off_par_%03d'%ctr
    uname_all.append(uname_rgb)




totalNum_units = len(mesopic_on_p) + len(mesopic_off_p)


mesopic_ids=mesopic_on_p+mesopic_off_p
mesopic_ids=np.array(mesopic_ids)

idx_allunits=mesopic_ids


In [ ]:

spikerate_grand = np.empty((0,totalNum_units,1))
idx_allunits= mesopic_ids                           
lightLevel_idx = 1


for lightLevel_idx in [1]:     #[0=0.3; 1=3; 2=30]
    select_lightLevel = lightLevel_text[lightLevel_idx]

    fname = os.path.join(path_raw,expDate+'_Noise_'+select_lightLevel+'.pkl')
    with open(fname,'rb') as f:
        params = pickle.load(f)


    ## PSTH ( bin size= 8ms)
    psth = params['psth']
    cluster_id = params['cluster_id']
    pre_pts = params['pre_pts']
    stim_pts = params['stim_pts']
    frame_rate = params['frame_rate'] # The precise update rate of the monitor computed from the timing frame flips.
    num_checkers_x = int(np.unique(params['numXChecks']))
    num_checkers_y = int(np.unique(params['numYChecks']))
    checkSize_um = 30

    t_frame = (1/frame_rate)*1000/2    # because spikerate is sampled at 120 but stim at 60Hz. So we will upsample stim later. So each frame is 8 ms
    
    binned_spikes = psth[:,:,pre_pts[0]:pre_pts[0]+int(params['numFrames'][0]*stride)]
    
    num_units_inDataset = psth.shape[0]

    # ---- stim
    idx = 1
    stim_frames = np.zeros((0,75,100))
    spikes = np.zeros((num_units_inDataset,0))
    for idx in tqdm(range(len(params['numXStixels'])), desc=''):
        frames_temp = get_spatial_noise_frames(
            int(params['numXStixels'][idx]),
            int(params['numYStixels'][idx]),
            int(params['numXChecks'][idx]),
            int(params['numYChecks'][idx]),
            params['chromaticClass'][idx],
            int(params['numFrames'][idx]),
            int(params['stepsPerStixel'][idx]),
            int(params['seed'][idx]),
            int(params['frameDwell'][idx]))
        
        # frames_temp[frames_temp>0] = 1
        # frames_temp[frames_temp<0] = 0
        if CONVERT_RSTAR == True:
            meanIntensity = lightLevel_mul[lightLevel_idx]
            frames_temp = applyLightIntensities(meanIntensity,frames_temp,t_frame)


        stim_frames = np.concatenate((stim_frames,frames_temp[:,:,:,select_colChan]))
        spikes_temp = binned_spikes[:,idx,:]
        spikes = np.concatenate((spikes,spikes_temp),axis=-1)
    
    
    stim_frames = np.reshape(stim_frames,(stim_frames.shape[0],stim_frames.shape[1]*stim_frames.shape[2]),order='F')
    
    stim_frames = np.repeat(stim_frames,stride,axis=0)
    assert stim_frames.shape[0] == spikes.shape[-1],'num of frames does not match num of spikes'


    # a = np.unique(stim_frames,axis=0)
    
    
    # ---- spike rates
    stimLength = stim_frames.shape[0]
    
    flips = np.arange(0,(stimLength+1)*t_frame,t_frame)
    
    numTrials = 1
    spikeRate_cells = np.empty((stimLength,totalNum_units,numTrials))
    spikeCounts_cells = np.empty((stimLength,totalNum_units,numTrials))
    spikeTimes_cells = np.empty((totalNum_units,numTrials),dtype='object')
    
    
    ctr_units = -1
    print("Total units to process:", len(idx_allunits))
    print("Cluster IDs available:", len(np.unique(cluster_id)))
    print("\nStarting unit processing...")
    U=0
    skipped_units = []
    found_units=[]
    for U in range(0, len(idx_allunits)):
        ctr_units += 1
        cluster_unit = idx_allunits[U]
        idx_unit_array = np.where(cluster_id==cluster_unit)[0]
        # if len(idx_unit_array) == 0:
        #     skipped_units.append(cluster_unit)
        #     print(f"Skipping unit {cluster_unit} - not found in cluster_id")
        #     continue  # Skip if cluster_unit is not found in cluster_id
        
        # found_units.append(cluster_unit)
        idx_unit = idx_unit_array[0]
        tr=0
        for tr in range(numTrials):
            startTime = 0#stimulus_start_times[tr]
            endTime = startTime + flips[-1]

            # using the PSTH 
            spikeCounts = spikes[idx_unit]
            spikeRate = MEA_spikerates_binned(spikeCounts,sig)
            # plt.plot(spikeRate)
        
            spikeRate_cells[:,ctr_units,tr] = spikeRate
            spikeCounts_cells[:,ctr_units,tr] = spikeCounts
            spikeTimes_cells[ctr_units,tr] = np.where(spikeCounts)[0]
            
            
            # Add visualization here
            fig, axes = plot_cell_response(
                cell_idx=idx_unit,
                spikes=spikes,
                psth=psth,
                cluster_id=cluster_id,
                sig=sig,
                t_frame=t_frame,
                frame_rate=frame_rate
            )
            plt.show()
        
    if NORM_RESP==True:
        rgb = np.squeeze(spikeRate_cells)
        rgb[rgb==0]=np.nan
        resp_median = np.nanmedian(rgb,axis=0)
        resp_norm = rgb/resp_median[None,:]
        resp_norm[np.isnan(resp_norm)] = 0
        resp_orig = spikeRate_cells
    else:
        resp_norm = spikeRate_cells
        resp_orig = spikeRate_cells
    
    stim_frames_train = (stim_frames,flips)
    spikeRate_train = (resp_norm,spikeCounts_cells,spikeTimes_cells)   # dims [stim_files][0=spikeRate,1=spikeCounts,2=spikeTimes]
    spikeRate_orig = spikeRate_cells
    spikeRate_median = resp_median
    
    spikerate_grand = np.concatenate((spikerate_grand,spikeRate_cells),axis=0)

    # ---- Save dataset
    with h5py.File(fname_save,'a') as f:
        try:
            f.create_dataset('expDate',data=np.array(expDate,dtype='bytes'))
            f.create_dataset('units',data=np.array(uname_all,dtype='bytes'))
        except:
            pass
    
        grp = f.create_group('/'+select_lightLevel+'/train')
        d = grp.create_dataset('stim_frames',data=stim_frames_train[0],compression='gzip')
        d.attrs['num_checkers_x'] = num_checkers_x
        d.attrs['num_checkers_y'] = num_checkers_y
        d.attrs['checkSize_um'] = checkSize_um
        d.attrs['t_frame'] = t_frame
        
        
        d = grp.create_dataset('flips_timestamp',data=stim_frames_train[1],compression='gzip')
        d.attrs['time_unit'] = 'ms' 
        
        d = grp.create_dataset('spikeRate',data=spikeRate_train[0],compression='gzip')
        d.attrs['bins'] = 'bin edges defined by dataset flips_timestamp'
        d.attrs['num_units'] = len(uname_all)
        d.attrs['sig'] = sig
        
        d = grp.create_dataset('spikeCounts',data=spikeRate_train[1],compression='gzip')
        d.attrs['bins'] = 'bin edges defined by dataset flip_times'
        d.attrs['num_units'] = len(uname_all)
        
        d = grp.create_dataset('spikeRate_orig',data=spikeRate_orig,compression='gzip')
        d.attrs['bins'] = 'bin edges defined by dataset flip_times'
        d.attrs['num_units'] = len(uname_all)

        d = grp.create_dataset('spikeRate_median',data=spikeRate_median,compression='gzip')
        d.attrs['bins'] = 'bin edges defined by dataset flip_times'
        d.attrs['num_units'] = len(uname_all)


        

spikerate_grand_flattened = np.moveaxis(spikerate_grand,0,-1)
spikerate_grand_flattened = spikerate_grand_flattened.reshape(spikerate_grand_flattened.shape[0],-1)
resp_median_grand = np.nanmedian(spikerate_grand_flattened,axis=-1)



with h5py.File(fname_save, 'a') as f:
    if 'spikerate_grand' in f:
        del f['spikerate_grand']  # Delete existing dataset if it exists
    if 'resp_median_grand' in f:
        del f['resp_median_grand']  # Delete existing dataset if it exists
    f.create_dataset('spikerate_grand', data=spikerate_grand_flattened, compression='gzip')
    f.create_dataset('resp_median_grand', data=resp_median_grand, compression='gzip')

In [ ]:
f = h5py.File(fname_save,'r')
a = f['mesopic']['train']['stim_frames']
print(a.shape)
f.close()

# %% STA
from model.featureMaps import decompose
f = h5py.File(fname_save,'r')
stim_frames_train = np.array(f['mesopic']['train']['stim_frames'])
spikeRate_train = np.array(f['mesopic']['train']['spikeRate'])
spikeRate_train = np.squeeze(np.array(f['mesopic']['train']['spikeCounts']))
f.close()

# %%
idx_data = np.arange(0,10000)
data_lim = 100000
stim = stim_frames_train[:data_lim]
stim = np.reshape(stim,(stim.shape[0],num_checkers_y,num_checkers_x),order='F')       
stim = stim-stim.mean()

idx_cell = 5
spikes = spikeRate_train[:,idx_cell][:data_lim]
spikes = np.where(spikes)[0]


num_spikes = len(spikes)
spikeCount = 0
nFrames =100
sta = np.zeros((nFrames,stim.shape[1],stim.shape[2]))

idx_start = np.where(spikes>nFrames)[0][0]
for i in tqdm(range(idx_start,num_spikes)):
    if i>spikes.shape[0]:
        break
    else:
        last_frame = spikes[i]
        first_frame = last_frame - nFrames
        
        sta = sta + stim[first_frame:last_frame,:,:]
        spikeCount+=1
sta = sta/spikeCount

spatial_feature, temporal_feature = decompose(sta)
# plt.plot(1,2,1)
# plt.imshow(spatial_feature,cmap='winter')
# plt.plot(1,2,2)
# plt.plot(temporal_feature)
# plt.title(f'RGC: {uname_all[idx_cell]}, Cell ID {idx_allunits[idx_cell]}')
# plt.show()


fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Plot spatial feature on the left subplot
ax1.imshow(spatial_feature, cmap='gray')
ax1.set_title('Spatial Feature')

# Plot temporal feature on the right subplot
ax2.plot(temporal_feature)
ax2.set_title('Temporal Feature')

# Add overall title for the figure
fig.suptitle(f'RGC: {uname_all[idx_cell]}, Cell ID {idx_allunits[idx_cell]}')

plt.tight_layout()
plt.show()